In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/


/content/drive/MyDrive


In [3]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz


--2025-02-02 04:37:58--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz.2’

cifar-10-python.tar 100%[===================>] 162.60M  45.7MB/s    in 4.0s    

2025-02-02 04:38:03 (40.5 MB/s) - ‘cifar-10-python.tar.gz.2’ saved [170498071/170498071]



In [4]:
!tar -xvzf cifar-10-python.tar.gz


cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [5]:
!ls cifar-10-batches-py


batches.meta  data_batch_2  data_batch_4  readme.html
data_batch_1  data_batch_3  data_batch_5  test_batch


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader
import wandb
import numpy as np
from tqdm import tqdm
import os
import pickle
import subprocess

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EPOCHS = 100
NUM_CLASSES = 10

In [ ]:
def download_and_extract_cifar10():
    # Download CIFAR-10 if not already downloaded
    if not os.path.exists('cifar-10-batches-py'):
        print("Downloading CIFAR-10 dataset...")
        subprocess.run(['wget', 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'])
        print("Extracting CIFAR-10 dataset...")
        subprocess.run(['tar', '-xzf', 'cifar-10-python.tar.gz'])
        subprocess.run(['rm', 'cifar-10-python.tar.gz'])  # Clean up tar file

In [ ]:

def load_batch(file_path):
    with open(file_path, 'rb') as f:
        batch = pickle.load(f, encoding='bytes')
    data = batch[b'data'].reshape(-1, 3, 32, 32)
    labels = batch[b'labels']
    return data, labels

In [ ]:
def load_cifar10():
    download_and_extract_cifar10()

    # Load training data
    train_data = []
    train_labels = []
    for i in range(1, 6):
        file_path = f'cifar-10-batches-py/data_batch_{i}'
        data, labels = load_batch(file_path)
        train_data.append(data)
        train_labels.extend(labels)

    train_data = np.concatenate(train_data)
    train_labels = np.array(train_labels)

    # Load test data
    test_data, test_labels = load_batch('cifar-10-batches-py/test_batch')

    return (train_data, train_labels), (test_data, np.array(test_labels))

In [ ]:
class CIFAR10Dataset(torch.utils.data.Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = torch.FloatTensor(data)
        self.labels = torch.LongTensor(labels)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.data[idx] / 255.0  # Normalize to [0, 1]
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

In [ ]:
# Data augmentation and normalization
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])


In [ ]:
# Load CIFAR-10 dataset
(train_data, train_labels), (test_data, test_labels) = load_cifar10()

In [ ]:
# Create datasets
full_train_dataset = CIFAR10Dataset(train_data, train_labels, transform_train)
test_dataset = CIFAR10Dataset(test_data, test_labels, transform_test)

In [ ]:
# Split training data into train and validation sets
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [ ]:


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                              stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                              stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                         stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

In [ ]:
class CIFAR10CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CIFAR10CNN, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        self.layer1 = self.make_layer(64, 2, stride=1)
        self.layer2 = self.make_layer(128, 2, stride=2)
        self.layer3 = self.make_layer(256, 2, stride=2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)

    def make_layer(self, out_channels, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(ResidualBlock(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in tqdm(train_loader, desc='Training'):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    accuracy = 100. * correct / total
    avg_loss = running_loss / len(train_loader)
    return avg_loss, accuracy

In [ ]:
def evaluate(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in tqdm(data_loader, desc='Evaluating'):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    accuracy = 100. * correct / total
    avg_loss = running_loss / len(data_loader)
    return avg_loss, accuracy

In [ ]:
def main():
    wandb.login(key="cc69bae954e1e13ab072d175549500283da08831")  # Ensure you replace "your-api-key" with your actual WandB API key

    wandb.init(project="cifar10-cnn", name="resnet-like-model", config={
        "batch_size": BATCH_SIZE,
        "learning_rate": LEARNING_RATE,
        "epochs": EPOCHS,
        "num_classes": NUM_CLASSES
    })

    model = CIFAR10CNN(NUM_CLASSES).to(device)
    wandb.watch(model, log="all", log_freq=10)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1,
                                                   patience=5, verbose=True)

    best_val_acc = 0
    for epoch in range(EPOCHS):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        wandb.log({
            'epoch': epoch,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'val_loss': val_loss,
            'val_acc': val_acc
        })

        # Print progress
        print(f'Epoch: {epoch+1}/{EPOCHS}')
        print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%')

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

        # Adjust learning rate
        scheduler.step(val_loss)

    # Load best model and evaluate on test set
    model.load_state_dict(torch.load('best_model.pth'))
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%')

    wandb.log({'test_acc': test_acc})
    wandb.finish()

if __name__ == '__main__':
    main()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
Evaluating: 100%|██████████| 79/79 [00:04<00:00, 17.99it/s]


Epoch: 1/100
Train Loss: 1.4768 | Train Acc: 45.55%
Val Loss: 1.2448 | Val Acc: 55.72%


Evaluating: 100%|██████████| 79/79 [00:04<00:00, 18.39it/s]


Epoch: 2/100
Train Loss: 1.0286 | Train Acc: 63.17%
Val Loss: 1.0624 | Val Acc: 61.82%


Evaluating: 100%|██████████| 79/79 [00:04<00:00, 18.42it/s]


...

Evaluating: 100%|██████████| 79/79 [00:04<00:00, 18.20it/s]
<ipython-input-37-247f6753c09b>:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.loa

Epoch: 100/100
Train Loss: 0.0274 | Train Acc: 99.28%
Val Loss: 0.2858 | Val Acc: 91.76%


Evaluating: 100%|██████████| 79/79 [00:03<00:00, 25.92it/s]

Test Loss: 0.3162 | Test Acc: 91.73%


epoch,▁▁▁▂▂▂▂▃▁▁▁▁▁▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇█
test_acc,▁
train_acc,▃▄▅▆▇▇▇▇▁▅▇▇▇▇▇█████████████████████████
train_loss,█▆▅▃▃▂▂▂▂▂▁▆▅▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▂▂▅▅▆▇▆▇▇▇▁▄▅▆▆▇████████████████████████
val_loss,▆█▄▄▄▃▂▃▃▂▂▂▆▄▄▃▃▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99
test_acc,91.73
train_acc,99.2775
train_loss,0.0274
val_acc,91.76


In [8]:
import wandb
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim

# Data augmentation and normalization
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Residual Block Definition
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                              stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                              stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                         stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

# CIFAR10CNN Model with Residual Blocks
class CIFAR10CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CIFAR10CNN, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        self.layer1 = self.make_layer(64, 2, stride=1)
        self.layer2 = self.make_layer(128, 2, stride=2)
        self.layer3 = self.make_layer(256, 2, stride=2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)

    def make_layer(self, out_channels, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(ResidualBlock(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Training and evaluation functions
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

def evaluate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

# Load CIFAR-10 dataset with data augmentation
def load_data(batch_size=128):
    train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

# Hyperparameters and device setup
LEARNING_RATES = [0.001, 0.0005, 0.0001]
OPTIMIZERS = ['adam', 'sgd']
EPOCHS = 10
NUM_CLASSES = 10  # CIFAR-10 has 10 classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Experiment function
def run_experiment(learning_rate, optimizer_name):
    wandb.init(project="cifar10-cnn",
               name=f"lr-{learning_rate}_opt-{optimizer_name}_batch-128",
               config={
                   "learning_rate": learning_rate,
                   "optimizer": optimizer_name,
                   "batch_size": 128,
                   "epochs": EPOCHS,
                   "num_classes": NUM_CLASSES
               })

    # Load dataset with batch size 128
    train_loader, val_loader, test_loader = load_data(batch_size=128)

    # Initialize model, loss function, and optimizer
    model = CIFAR10CNN(NUM_CLASSES).to(device)
    wandb.watch(model, log="all", log_freq=10)

    criterion = nn.CrossEntropyLoss()

    if optimizer_name == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1,
                                                     patience=5, verbose=True)

    best_val_acc = 0
    for epoch in range(EPOCHS):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)

        wandb.log({
            'epoch': epoch,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'val_loss': val_loss,
            'val_acc': val_acc
        })

        # Print progress
        print(f'Epoch: {epoch+1}/{EPOCHS}')
        print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%')

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            model_save_path = f'best_model_{wandb.run.name}.pth'
            torch.save(model.state_dict(), model_save_path)

            # Upload the model directly to W&B as an artifact
            artifact = wandb.Artifact("best_model", type="model")
            artifact.add_file(model_save_path)
            wandb.log_artifact(artifact)

        # Adjust learning rate
        scheduler.step(val_loss)

    # Load best model and evaluate on test set
    model.load_state_dict(torch.load(f'best_model_{wandb.run.name}.pth'))
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%')

    wandb.log({'test_acc': test_acc})
    wandb.finish()


# Main function to run all experiments
def main():
    # First, log in to W&B (only needed once)
    wandb.login(key="cc69bae954e1e13ab072d175549500283da08831")  # Replace with your actual WandB API key

    # Run experiments by varying learning rate and optimizer
    for lr in LEARNING_RATES:
        for optimizer_name in OPTIMIZERS:
            run_experiment(lr, optimizer_name)

if __name__ == '__main__':
    main()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch: 1/10
Train Loss: 1.4034 | Train Acc: 48.35%
Val Loss: 1.0979 | Val Acc: 60.90%
Epoch: 2/10
Train Loss: 0.9325 | Train Acc: 66.94%
Val Loss: 0.8336 | Val Acc: 70.65%
Epoch: 3/10
Train Loss: 0.7425 | Train Acc: 73.92%
Val Loss: 0.7847 | Val Acc: 72.56%
Epoch: 4/10
Train Loss: 0.6216 | Train Acc: 78.28%
Val Loss: 0.8119 | Val Acc: 72.14%
Epoch: 5/10
Train Loss: 0.5371 | Train Acc: 81.29%
Val Loss: 0.5802 | Val Acc: 79.87%
Epoch: 6/10
Train Loss: 0.4757 | Train Acc: 83.55%
Val Loss: 0.5092 | Val Acc: 82.24%
Epoch: 7/10
Train Loss: 0.4337 | Train Acc: 85.20%
Val Loss: 0.4266 | Val Acc: 85.27%
Epoch: 8/10
Train Loss: 0.3867 | Train Acc: 86.65%
Val Loss: 0.3545 | Val Acc: 87.60%
Epoch: 9/10
Train Loss: 0.3579 | Train Acc: 87.58%
Val Loss: 0.3713 | Val Acc: 87.03%
Epoch: 10/10
Train Loss: 0.3318 | Train Acc: 88.44%
Val Loss: 0.3048 | Val Acc: 89.42%


<ipython-input-8-a502604e1330>:213: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_model_{wandb.run.name}.pth'))


...

epoch,▁▂▃▃▄▅▆▆▇█
test_acc,▁
train_acc,▁▃▅▆▆▇▇▇██
train_loss,█▆▄▄▃▂▂▂▁▁
val_acc,▁▂▄▅▆▇▇███
val_loss,█▇▅▄▃▃▂▁▁▁
epoch,9
test_acc,72.35
train_acc,74.398
train_loss,0.73207
val_acc,74.032


Files already downloaded and verified
Files already downloaded and verified
Epoch: 1/10
Train Loss: 1.4173 | Train Acc: 48.12%
Val Loss: 1.2082 | Val Acc: 57.04%
Epoch: 2/10
Train Loss: 1.0188 | Train Acc: 63.60%
Val Loss: 0.9142 | Val Acc: 67.75%
Epoch: 3/10
Train Loss: 0.8333 | Train Acc: 70.57%
Val Loss: 0.7837 | Val Acc: 72.56%
Epoch: 4/10
Train Loss: 0.7117 | Train Acc: 75.37%
Val Loss: 0.7015 | Val Acc: 75.71%
Epoch: 5/10
Train Loss: 0.6322 | Train Acc: 78.24%
Val Loss: 0.6102 | Val Acc: 78.95%
Epoch: 6/10
Train Loss: 0.5715 | Train Acc: 80.21%
Val Loss: 0.5348 | Val Acc: 81.76%
Epoch: 7/10
Train Loss: 0.5207 | Train Acc: 82.16%
Val Loss: 0.6236 | Val Acc: 78.32%
Epoch: 8/10
Train Loss: 0.4824 | Train Acc: 83.62%
Val Loss: 0.4435 | Val Acc: 85.08%
Epoch: 9/10
Train Loss: 0.4434 | Train Acc: 84.72%
Val Loss: 0.4948 | Val Acc: 82.55%
Epoch: 10/10
Train Loss: 0.4139 | Train Acc: 85.86%
Val Loss: 0.4324 | Val Acc: 85.26%
Test Loss: 0.5651 | Test Acc: 81.68%


epoch,▁▂▃▃▄▅▆▆▇█
test_acc,▁
train_acc,▁▄▅▆▇▇▇███
train_loss,█▅▄▃▃▂▂▁▁▁
val_acc,▁▄▅▆▆▇▆█▇█
val_loss,█▅▄▃▃▂▃▁▂▁
epoch,9
test_acc,81.68
train_acc,85.864
train_loss,0.41392
val_acc,85.258


Files already downloaded and verified
Files already downloaded and verified
Epoch: 1/10
Train Loss: 2.0787 | Train Acc: 24.31%
Val Loss: 1.8914 | Val Acc: 30.21%
Epoch: 2/10
Train Loss: 1.8220 | Train Acc: 33.24%
Val Loss: 1.7519 | Val Acc: 36.16%
Epoch: 3/10
Train Loss: 1.7101 | Train Acc: 37.18%
Val Loss: 1.6535 | Val Acc: 39.40%
Epoch: 4/10
Train Loss: 1.6228 | Train Acc: 40.50%
Val Loss: 1.5832 | Val Acc: 42.21%
Epoch: 5/10
Train Loss: 1.5516 | Train Acc: 43.40%
Val Loss: 1.5026 | Val Acc: 44.99%
Epoch: 6/10
Train Loss: 1.4808 | Train Acc: 46.06%
Val Loss: 1.4528 | Val Acc: 47.27%
Epoch: 7/10
Train Loss: 1.4178 | Train Acc: 48.39%
Val Loss: 1.3812 | Val Acc: 49.62%
Epoch: 8/10
Train Loss: 1.3556 | Train Acc: 51.32%
Val Loss: 1.3673 | Val Acc: 50.63%
Epoch: 9/10
Train Loss: 1.3107 | Train Acc: 52.70%
Val Loss: 1.2853 | Val Acc: 53.62%
Epoch: 10/10
Train Loss: 1.2709 | Train Acc: 54.19%
Val Loss: 1.3294 | Val Acc: 52.20%
Test Loss: 1.2945 | Test Acc: 53.52%


epoch,▁▂▃▃▄▅▆▆▇█
test_acc,▁
train_acc,▁▃▄▅▅▆▇▇██
train_loss,█▆▅▄▃▃▂▂▁▁
val_acc,▁▃▄▅▅▆▇▇██
val_loss,█▆▅▄▄▃▂▂▁▂
epoch,9
test_acc,53.52
train_acc,54.19
train_loss,1.27086
val_acc,52.202
